# Preprocessing

### Setting Virtual Env and Installing dependencies

In [15]:
!source env/bin/activate
!pip3 install boto3
!pip3 install pandas
!pip3 install pyarrow


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: /usr/local/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: /usr/local/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: /usr/local/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip


### Prepare data

In [16]:
import time
import boto3
from time import sleep
import pandas as pd
import json
from datetime import datetime

In [17]:
original_data = pd.read_csv("data/test-2.csv")
original_data.head(10)

,USER_ID,ITEM_ID,EVENT_TYPE,video_duration,duration_watched,Liked,Commented,Shared,TIMESTAMP
0,4d9389046a07a79e6f73941cbfe05406,3f44d737119484cb06dc8074cfd6ba99,Watched,19,7,0,1,1,1705564972
1,de9f506fd891d7d98b0e93bcb99d464f,cea477054ab26d3e85468347d2b339da,Watched,55,9,0,1,1,1705590871
2,a2288579b87050b04d5e0188a96a54fb,fc51ec638fad4a4f2346b2b8c3fb9a1c,Watched,16,5,0,1,1,1705480483
3,263a4644ea5b873ba35b14c502c7010b,58d5743b213dcef8a3d931217ac1c687,Watched,13,13,0,1,0,1705664039
4,4001bdccd34e8fdfd564bd1078d994ca,23070d03a2b374b078414bf73666a4df,Watched,6,5,0,1,1,1705793960
5,5107502f985aa0b73854b067bc41f4b3,086b999c965e7d784a7cc7dee3187406,Watched,36,21,0,0,0,1705508032
6,a2288579b87050b04d5e0188a96a54fb,c2a9bb601fd2f559f67db470202fdd1a,Watched,40,17,1,1,1,1705128846
7,f468121171bac06d7500fc1f937c4b11,086b999c965e7d784a7cc7dee3187406,Watched,78,24,0,1,0,1705526681
8,f468121171bac06d7500fc1f937c4b11,b6151a57fd74de9ee3bd0ca09bb82b45,Watched,73,18,0,0,0,1705284098
9,245f9e670de492e964f2f71e4201b345,3f44d737119484cb06dc8074cfd6ba99,Watched,43,30,1,0,1,1705672332


In [18]:
print("Unique_users:" + str(len(original_data['USER_ID'].unique())))

Unique_users:25


In [19]:
print("Unique_items:" + str(len(original_data['ITEM_ID'].unique())))

Unique_items:37


In [20]:
print("Number of interactions:" + str(len(original_data.index)))

Number of interactions:20000


### Check Timestamp for validity

In [21]:
arb_time_stamp = original_data.iloc[50]['TIMESTAMP']
formatted_time = datetime.utcfromtimestamp(arb_time_stamp).strftime('%Y-%m-%d %H:%M:%S')
arb_time_stamp, formatted_time

(1705514666, '2024-01-17 18:04:26')

In [22]:
# Create a new dataframe for cleaned data:
cleaned_data = original_data.copy()
cleaned_data = cleaned_data[cleaned_data['video_duration'].notna() & cleaned_data['duration_watched'].notna()]
cleaned_data

,USER_ID,ITEM_ID,EVENT_TYPE,video_duration,duration_watched,Liked,Commented,Shared,TIMESTAMP
0,4d9389046a07a79e6f73941cbfe05406,3f44d737119484cb06dc8074cfd6ba99,Watched,19,7,0,1,1,1705564972
1,de9f506fd891d7d98b0e93bcb99d464f,cea477054ab26d3e85468347d2b339da,Watched,55,9,0,1,1,1705590871
2,a2288579b87050b04d5e0188a96a54fb,fc51ec638fad4a4f2346b2b8c3fb9a1c,Watched,16,5,0,1,1,1705480483
3,263a4644ea5b873ba35b14c502c7010b,58d5743b213dcef8a3d931217ac1c687,Watched,13,13,0,1,0,1705664039
4,4001bdccd34e8fdfd564bd1078d994ca,23070d03a2b374b078414bf73666a4df,Watched,6,5,0,1,1,1705793960
...,...,...,...,...,...,...,...,...,...
19995,de9f506fd891d7d98b0e93bcb99d464f,93eca6b49eb656f26536572e588be8ff,Watched,58,43,1,1,1,1705718510
19996,16f88ca54189cd79ecb8cb1c43c36dd4,40fc2e98607e6b6448693cfa448f909b,Watched,39,37,1,0,0,1705391868
19997,22e7ff9d170ec82d8209abe6379a7d80,26568621ecebca2a253a7010f528abf5,Watched,78,65,0,1,1,1705276593
19998,07907c62615bb3154efd910e84b5c279,cea477054ab26d3e85468347d2b339da,Watched,51,15,1,0,0,1705778502


In [23]:
# Define the threshold percentage
THRESHOLD_PERCENTAGE = 50 

watched_df = cleaned_data.copy()
# Calculate the threshold duration based on the percentage
watched_df['threshold_duration'] = watched_df['video_duration'] * (THRESHOLD_PERCENTAGE / 100)
watched_df = watched_df[watched_df['duration_watched'] >= watched_df['threshold_duration']]
watched_df = watched_df[['USER_ID', 'ITEM_ID', 'EVENT_TYPE', 'TIMESTAMP']]
watched_df['EVENT_TYPE'] = 'watched'
watched_df.head()

,USER_ID,ITEM_ID,EVENT_TYPE,TIMESTAMP
3,263a4644ea5b873ba35b14c502c7010b,58d5743b213dcef8a3d931217ac1c687,watched,1705664039
4,4001bdccd34e8fdfd564bd1078d994ca,23070d03a2b374b078414bf73666a4df,watched,1705793960
5,5107502f985aa0b73854b067bc41f4b3,086b999c965e7d784a7cc7dee3187406,watched,1705508032
9,245f9e670de492e964f2f71e4201b345,3f44d737119484cb06dc8074cfd6ba99,watched,1705672332
10,53ffd4c31493227741ddc006ddb26510,4b40d87c36793bbf370868135e1a9907,watched,1705758630


In [24]:
clicked_df = cleaned_data.copy()

# Calculate the threshold duration based on the percentage
clicked_df['threshold_duration'] = clicked_df['video_duration'] * (THRESHOLD_PERCENTAGE / 100)
clicked_df = clicked_df[(clicked_df['duration_watched'] < clicked_df['threshold_duration']) & (clicked_df['duration_watched'] > 0)]
clicked_df = clicked_df[['USER_ID', 'ITEM_ID', 'EVENT_TYPE', 'TIMESTAMP']]
clicked_df['EVENT_TYPE'] = 'clicked'
clicked_df.head()

,USER_ID,ITEM_ID,EVENT_TYPE,TIMESTAMP
0,4d9389046a07a79e6f73941cbfe05406,3f44d737119484cb06dc8074cfd6ba99,clicked,1705564972
1,de9f506fd891d7d98b0e93bcb99d464f,cea477054ab26d3e85468347d2b339da,clicked,1705590871
2,a2288579b87050b04d5e0188a96a54fb,fc51ec638fad4a4f2346b2b8c3fb9a1c,clicked,1705480483
6,a2288579b87050b04d5e0188a96a54fb,c2a9bb601fd2f559f67db470202fdd1a,clicked,1705128846
7,f468121171bac06d7500fc1f937c4b11,086b999c965e7d784a7cc7dee3187406,clicked,1705526681


### Combine the two Interactions and save as CSV for Personalize use

In [30]:
interaction_df = clicked_df.copy()
interaction_df = pd.concat([clicked_df, watched_df], ignore_index=True)
interaction_df.sort_values("TIMESTAMP", axis=0, ascending=True, inplace=True, na_position="last")
interaction_df

,USER_ID,ITEM_ID,EVENT_TYPE,TIMESTAMP
15153,263a4644ea5b873ba35b14c502c7010b,58d5743b213dcef8a3d931217ac1c687,watched,1705104074
8750,6815c1020771effac81e5295cf9c0553,0dde45f98279700aa649783b45c391c0,clicked,1705104123
19712,af9684fd5ed1d0efaf6074aafd159881,3f44d737119484cb06dc8074cfd6ba99,watched,1705104155
10908,6815c1020771effac81e5295cf9c0553,58d5743b213dcef8a3d931217ac1c687,watched,1705104174
15119,a2288579b87050b04d5e0188a96a54fb,79d00fee3b1572fc49818ce2f9ac859f,watched,1705104194
...,...,...,...,...
1352,91b6ce1027172cf5292ebddbe6d26407,076bb9628c8bd416f481a363d10b84a0,clicked,1705794859
684,3adcf32f5562d8568d613b325044405c,70ca3b97f22088ee6bc4bc22e734ab79,clicked,1705794905
1332,f468121171bac06d7500fc1f937c4b11,58d5743b213dcef8a3d931217ac1c687,clicked,1705795064
2617,de9f506fd891d7d98b0e93bcb99d464f,c2a9bb601fd2f559f67db470202fdd1a,clicked,1705795102


In [40]:
interaction_filename = "interaction.csv"
interaction_df.to_csv(f"data/{interaction_filename}", index=False, float_format="%.2f")

### Create an IAM Role

In [52]:
# Check and ensure AWS credentials are correctly configured
# Verify IAM permissions for the user/role associated with the credentials

# Initialize IAM client
session = boto3.session.Session(profile_name='default')
iam = session.client('iam')

role_name = "PersonalizeRole"

# Define assume role policy document
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

# Create IAM role with the specified name and assume role policy document
create_role_response = iam.create_role(
    RoleName=role_name,
    AssumeRolePolicyDocument=json.dumps(assume_role_policy_document)
)

# Attach Amazon Personalize full access policy
policy_arn_personalize = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName=role_name,
    PolicyArn=policy_arn_personalize
)

# Attach Amazon S3 full access policy
policy_arn_s3 = "arn:aws:iam::aws:policy/AmazonS3FullAccess"
iam.attach_role_policy(
    RoleName=role_name,
    PolicyArn=policy_arn_s3
)

# Wait for a minute to allow IAM role policy attachment to propagate
time.sleep(60)

# Get the ARN of the created role
role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

arn:aws:iam::219702648416:role/PersonalizeRole


### Validating and Importing Item Metadata

In [53]:
original_data = pd.read_csv("data/rG-SClckUS6ydztVjFWEg.csv")
original_data

,sno,item_id,genres,childlabel,creation_timestamp
0,1,6e0e010dee526f9d7aab46b2e4682a9f,"Outdoors,Food,Meat,Bread,Pancake,Cooking,Perso...","nature,outdoors,food,food presentation,meal,me...",1705566242
1,2,b8b3ce3b74093f68faa156731b941a10,"Person,Body Part,Head,Adult,Male,Face,Food,Wea...","adult,arm,body part,face,head,male,man,person,...",1705566353
2,3,3eefb39a416b942cc89fba599fdb2f5c,"Nature,Outdoors,Land,Plant,Tree,Vegetation,Pat...","land,nature,outdoors,path,plant,rainforest,tra...",1705566426
3,4,9c548e70860e94fcaf54d2758c0ba29a,"Plant,Field,Nature,Outdoors,Person,Adult,Male,...","field,grass,grassland,land,male,nature,outdoor...",1705566475
4,5,26cbcb970c9963d063f7d587fe7ff5a9,"Outdoors,Nature,Water,Person,Adult,Female,Land...","nature,outdoors,person,human,scenery,water,wat...",1705566568
...,...,...,...,...,...
72,73,1c80ab421aaf309cbad642147b4112b5,"Outdoors,Architecture,Nature,Water,Building,Urban","aerial view,bridge,outdoors,road,architecture,...",1705730860
73,74,864f69ce13deace55f5dda5c71d4e8a3,"Outdoors,Architecture,Nature,Water,Building,Urban","aerial view,bridge,outdoors,road,architecture,...",1705738594
74,75,1818e44ced0eeaa5676393aa4ce97a91,"Outdoors,Architecture,Building,Urban,Box,Natur...","aerial view,bridge,outdoors,road,architecture,...",1705738808
75,76,98b9503dfe2b76b8bf06e4c787a45034,"Person,Electronics,Hardware,Computer,Computer ...","adult,computer,computer hardware,computer keyb...",1705747438


In [54]:
original_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   sno                 77 non-null     int64 
 1   item_id             77 non-null     object
 2   genres              77 non-null     object
 3   childlabel          77 non-null     object
 4   creation_timestamp  77 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 3.1+ KB


In [66]:
item_metadata_df = original_data.copy()
item_metadata_df = item_metadata_df[['item_id', 'genres', 'creation_timestamp']]
item_metadata_df

,item_id,genres,creation_timestamp
0,6e0e010dee526f9d7aab46b2e4682a9f,"Outdoors,Food,Meat,Bread,Pancake,Cooking,Perso...",1705566242
1,b8b3ce3b74093f68faa156731b941a10,"Person,Body Part,Head,Adult,Male,Face,Food,Wea...",1705566353
2,3eefb39a416b942cc89fba599fdb2f5c,"Nature,Outdoors,Land,Plant,Tree,Vegetation,Pat...",1705566426
3,9c548e70860e94fcaf54d2758c0ba29a,"Plant,Field,Nature,Outdoors,Person,Adult,Male,...",1705566475
4,26cbcb970c9963d063f7d587fe7ff5a9,"Outdoors,Nature,Water,Person,Adult,Female,Land...",1705566568
...,...,...,...
72,1c80ab421aaf309cbad642147b4112b5,"Outdoors,Architecture,Nature,Water,Building,Urban",1705730860
73,864f69ce13deace55f5dda5c71d4e8a3,"Outdoors,Architecture,Nature,Water,Building,Urban",1705738594
74,1818e44ced0eeaa5676393aa4ce97a91,"Outdoors,Architecture,Building,Urban,Box,Natur...",1705738808
75,98b9503dfe2b76b8bf06e4c787a45034,"Person,Electronics,Hardware,Computer,Computer ...",1705747438


In [67]:
item_metadata_df.dtypes

item_id               object
genres                object
creation_timestamp     int64
dtype: object

In [68]:
item_metadata_df.rename(columns = {'item_id': 'ITEM_ID', 'genres': 'GENRES', 'creation_timestamp': 'CREATION_TIMESTAMP'}, inplace=True)

In [69]:
item_metadata_df

,ITEM_ID,GENRES,CREATION_TIMESTAMP
0,6e0e010dee526f9d7aab46b2e4682a9f,"Outdoors,Food,Meat,Bread,Pancake,Cooking,Perso...",1705566242
1,b8b3ce3b74093f68faa156731b941a10,"Person,Body Part,Head,Adult,Male,Face,Food,Wea...",1705566353
2,3eefb39a416b942cc89fba599fdb2f5c,"Nature,Outdoors,Land,Plant,Tree,Vegetation,Pat...",1705566426
3,9c548e70860e94fcaf54d2758c0ba29a,"Plant,Field,Nature,Outdoors,Person,Adult,Male,...",1705566475
4,26cbcb970c9963d063f7d587fe7ff5a9,"Outdoors,Nature,Water,Person,Adult,Female,Land...",1705566568
...,...,...,...
72,1c80ab421aaf309cbad642147b4112b5,"Outdoors,Architecture,Nature,Water,Building,Urban",1705730860
73,864f69ce13deace55f5dda5c71d4e8a3,"Outdoors,Architecture,Nature,Water,Building,Urban",1705738594
74,1818e44ced0eeaa5676393aa4ce97a91,"Outdoors,Architecture,Building,Urban,Box,Natur...",1705738808
75,98b9503dfe2b76b8bf06e4c787a45034,"Person,Electronics,Hardware,Computer,Computer ...",1705747438


In [70]:
item_meta_filename = "item-meta.csv"
item_metadata_df.to_csv(f"data/{item_meta_filename}", index=False, float_format="%.2f")